# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали.

Построим модель с максимально большим значением *accuracy*. Задача - довести долю правильных ответов по крайней мере до 0.75. Проверить *accuracy* на тестовой выборке.

## Откроем и изучим файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('users_behavior.csv')
print(df.head())
df.shape

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0


(3214, 5)

Мы импортировали часть от общего числа требуемых для анализа библиотек и загрузили данные. Сейчас посмотрим на их состояние и если всё ОК, сможем приступать к построению моделей

Мы решаем задачу классификации с учителем , где тарифу "Ультра" это 1, а "Смарт" это 0.

In [3]:
df.info() # видим что нет пропусков

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Данные были ранее обработаны и мы убедились что предобработка не нужна. Приступим к построению моделей. 

## Разобьем данные на выборки

In [4]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.40, random_state=12345) # отделим 40% данных для валидационной выборки
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.50, random_state=12345)
# разделим 1 к 1 валидационную выборку чтобы вычленить тестовую.

print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape) # проверим распределение

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


## Исследуем модели

1) Сначала посмотрим как сработает случайный лес

In [5]:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train,target_train) # обучим модель на тренировочной выборке
    result = model.score(features_valid,target_valid) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_model = model # сохраним наилучшую модель
        best_result = result #  сохраним наилучшее значение метрики accuracy на валидационных данных
print("Accuracy наилучшей модели на валидационной выборке случайного леса:", best_result)

Accuracy наилучшей модели на валидационной выборке случайного леса: 0.7853810264385692


Получили Accuracy на 0.79, так что мы сразу подобрали устраивающий нас вариант. Но попробуем что нибудь еще.

2) Попробуем логистическую регрессию

In [6]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train) 
result = model.score(features_valid, target_valid) 

print("Accuracy логистической регрессии на валидационной выборке:", result)

Accuracy логистической регрессии на валидационной выборке: 0.7107309486780715


Результат Accuracy 0.71 хуже предыдущего

3) Попробуем дерево решений , выведем на экран разные значения Accuracy для гиперпараметра max_depth. 

In [7]:
for depth in range(1,6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    
    print("Глубина уровня цикла =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

Глубина уровня цикла = 1 : 0.7542768273716952
Глубина уровня цикла = 2 : 0.7822706065318819
Глубина уровня цикла = 3 : 0.7853810264385692
Глубина уровня цикла = 4 : 0.7791601866251944
Глубина уровня цикла = 5 : 0.7791601866251944


Результат Accuracy хуже предыдущего нас так же устраивают, отметим что третий уровень совпадает со случайным лесом

Промежуточный вывод: лучше всего себя показала модель номер 1 случайный лес с результатом точности 0.7853810264385692 для валидационной выбор

## Проверим модель на тестовой выборке

In [8]:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train,target_train) 
    result = model.score(features_test,target_test) 
    if result > best_result:
        best_model = model 
        best_result = result 
print("Accuracy наилучшей модели на тестовой выборке случайного леса:", best_result)

Accuracy наилучшей модели на тестовой выборке случайного леса: 0.7869362363919129


Результат даже чуть лучше чем на выборке валидации

## Проверим модели на адекватность

In [9]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
DummyClassifier(strategy='most_frequent')
dummy_clf.predict(features_train)
dummy_clf.score(features_train, target_train)

0.6924273858921162

Дамми скор показывает значение в 0.69, это означает что мы можем быть довольны нашей моделью так как она показывает 0.79 качество предсказания

## Итоговый вывод

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобилась — мы её уже сделали.

Построим модель с максимально большим значением accuracy. Нужно довести долю правильных ответов по крайней мере до 75%.

1) Мы предварительно оценили данные. Ранее с ними уже была проведена работа, поэтому мы знаем что в них нет пропусков и можно приступать к работе.
Перед нами стоит задача провести машинное обучение алгоритма с учителем по классификации данных, где присутствует 2 класса тарифа - "Ультра" и "Смарт"

2) Мы разбили данные на выборки, выделили 60% под обучение, 20% для валидации модели и 20% я финального теста работоспособности обученной модели.

3) Мы провели исследование, чтобы понять какая модель машинного обучения лучше подойдет для наших целей. Получили следующие результаты:
 - Случайный лес показал 79% точности
 - Логистическую регрессия показала 71% точности
 - Дерево решений с изменением гиперпараметра для поиска лучшего результата так же показало точность в 79%

Мы приняли решение остановиться на модели случайного леса. 

4) Проверили модель случайного леса на тестовой выборке. Получили подтверждение результата точности в 79%. 

5) Проверили модель на адекватность. Мы выяснили что самый частый вариант классификации из двух составляет 69% от общего числа. Это говорит нам, что наша модель обучения по точности выше чем выбор в "слепом случае" и значит она может принести эффективную пользу для бизнеса. 

Как итог мы получили и проверили модель для перевода клиентов на новые тарифы. 